<a href="https://colab.research.google.com/github/AjmalSarwary/BRENT-Model/blob/main/brent_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/my_packages')
import api_alphavantage as av

In [ ]:
import requests
import json
import pandas as pd

def alphavantage_get( symbol,dfrom, dto,interval,api_key):
    
    '''retrieves time series of prices for function in range dfrom to dto at interval with api_key'''
    
    interval = interval
    dfrom=dfrom
    dto=dto
    api_key=api_key
    
    url = f'https://www.alphavantage.co/query?function={symbol}&interval={interval}&apikey={api_key}&outputsize=full'
    
    r = requests.get(url)
    data = r.json()
    
    df = pd.DataFrame(data)
    
    df['date'] = df['data'].apply(lambda x: x['date'])
    df['value'] = df['data'].apply(lambda x: x['value'])

    df.drop('data', axis=1, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    # filter data to specified time period
    df = df[(df.index >= dfrom) & (df.index <= dto)]

    #error message suggested that there is a string '.' ; ->locate and remove string
    df = df[df['value'] != '.']
    
    df.value = df['value'].astype(float)
    
    return df
    
    

In [ ]:
def moving_avs(dataframe):
    
    df=dataframe
    
    # make sure type is correct
    df['value'] = df.value.astype(float)
    
    # calculate weekly moving average
    weekly_ma = df['value'].rolling('7D').mean()
    df['weekly_ma'] = weekly_ma

    # calculate quaterly moving average
    yearly_ma = df['value'].rolling('365D').mean()
    df['yearly_ma'] = yearly_ma
    
    return df

In [ ]:
#plot dataframe
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd


def plotter(dataframe,symbol, moving=0,amount='', footnote=''):
    
    df=dataframe
    
    # plot the data
    
    
    
    if moving == 1:

       # create a new DataFrame with the moving averages

        df_ma = pd.concat([df['value'], df.weekly_ma, df.yearly_ma], axis=1)

        # Create the plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_ma.index, y=df_ma['weekly_ma'], name='Weekly Moving Average', opacity=0.3))
        fig.add_trace(go.Scatter(x=df_ma.index, y=df_ma['yearly_ma'], name='Yearly Moving Average'))

        # title and axis labels
        fig.update_layout(title=symbol, xaxis_title='Date', yaxis_title=f'in {amount} (USD)', 
                          legend_title='Moving Average', plot_bgcolor='  #cce6ff',paper_bgcolor=' #e6f2ff')

        # footnote
        fig.add_annotation(xref='paper', yref='paper', x=-0.1, y=-0.17, text=footnote, showarrow=False,  font=dict(
                family='Arial',
                size=9,
                
            ))

        
        fig.show()
        
    else: 

        fig = go.Figure() 
        fig.add_trace(go.Scatter(x=dataframe.index, y=dataframe.value, name='Global GDP'))
        fig.update_layout(title=symbol, xaxis_title='Date', yaxis_title=f'in {amount} (USD)', 
                          legend_title='symbol', plot_bgcolor='  #cce6ff',paper_bgcolor=' #e6f2ff')
        fig.add_annotation(xref='paper', yref='paper', x=-0.1, y=-0.17, text=footnote, showarrow=False,  font=dict(
                family='Arial',
                size=9,
                
            ))
        fig.show()


In [ ]:
#alphavantage_get entries
#call alphavantage_get(symbol,dfrom,dto, intervel, api_key)

#api key
api_key = av

#symbol
symbols=['BRENT','ALL_COMMODITIES']

# daily interval
interval = 'daily' 

#time period
start_date = '2005-01-01'
end_date = '2015-12-31'


In [ ]:
# get all_commodidities from alphavantage
comod = alphavantage_get(symbols[1], start_date, end_date, interval, api_key)
comod = moving_avs(comod)
amount=''
footnote='Source: International Monetary Fund (IMF Terms of Use), Global Price Index of All Commodities, retrieved from FRED, Federal Reserve Bank of St. Louiss; data accessed through AlphaVantage API.'
plotter(comod,'GLOBAL COMMODITIES',1,amount,footnote)


In [ ]:
#get brent data from alphavantage and plot
brent = alphavantage_get(symbols[0], start_date, end_date, interval, api_key)
brent = moving_avs(brent)
amount=''
footnote='Source: U.S. Energy Information Administration, Crude Oil Prices: Brent - Europe, retrieved from FRED, Federal Reserve Bank of St. Louis; Alpha Vantage API, accessed on 2023-04-05.'
plotter(brent,'BRENT-Europe',1,amount, footnote)

In [ ]:
!mkdir /content/drive/MyDrive/datasets

In [ ]:
from google.colab import files
import os

# Create directory if it doesn't exist
if not os.path.exists('/content/drive/MyDrive/datasets'):
    os.mkdir('/content/drive/MyDrive/datasets')

# Upload file
uploaded = files.upload()

# Get filename and content of uploaded file
filename, content = next(iter(uploaded.items()))

# Write content to file
with open('/content/drive/MyDrive/datasets/' + filename, 'wb') as f:
    f.write(content)

Saving gdp_wb_2005_2015.csv to gdp_wb_2005_2015 (6).csv


In [ ]:
import numpy as np

ggdp = pd.read_csv('/content/drive/MyDrive/datasets/gdp_wb_2005_2015.csv')

#Transform and format global gdp

ggdp['date']=ggdp['Year']
ggdp=ggdp.drop(columns=['Year'])
ggdp.set_index('date', inplace=True)

ggdp['value']=np.divide(ggdp['\tGDP (current US$)'],1e11)
ggdp = ggdp.drop(columns=['\tGDP (current US$)'])

ggdp.head(1)

,value
date,
2005,48.136604


In [ ]:
amount='Trillion'
footnote='Source: World Bank'
plotter(ggdp, 'Global GDP',0,amount, footnote )

# Neuer Abschnitt